# Fremtidsrettet data
### Ved hjelp av API


Gruppen har brukt data fra [developer.yr.no](https://api.met.no/weatherapi/locationforecast/2.0/documentation). Daten viser vær for gitte plasser i den kommende uken. Den inkluderer blant annet temperatur, vind fart og lufttrykk. Gruppen har valgt å bruke en API fra nettsiden som støtter JSON-filer.
Daten viser være for gitte plasser i den kommende uken. 


In [26]:
import pandas as pd
import requests

Under har gruppen lagt tilgjengelig noen koordinater som kan være enkle å bruke.
Her er link til [Google Maps](https://www.google.com/maps) dersom du ønsker å hente inn egne koordinater. 

In [ ]:
#Breddegrad og lengdegrad er hentet fra Google Maps
locations = [
    ("Stryn", 61.904275, 6.715641),
    ("Paris", 48.854289, 2.342042),
    ("London", 51.501814, -0.140605),
    ("Cape Town", -33.922348, 18.424031),
    ("New York", 40.710676, -74.006219)
]

#Lager en oversiktstabell
data = []
for name, lat, lon in locations:
    data.append([name, lat, lon])

df = pd.DataFrame(data, columns=["Plass", "Breddegrad", "Lengdegrad"])
df.head()


,Plass,Breddegrad,Lengdegrad
0,Stryn,61.904275,6.715641
1,Paris,48.854289,2.342042
2,London,51.501814,-0.140605
3,Cape Town,-33.922348,18.424031
4,New York,40.710676,-74.006219


# Databehandling
I kodene under blir det sjekket om det er noen hull i datafilene som blir lastet inn.

In [37]:
def make_weatherJSON(lat, lon):
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    headers = {'User-Agent': 'my-weather-app/1.0'}  #nødvendig for Yr API
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"API call failed with status code {response.status_code}")
    
    data = response.json()
    return data

make_weatherJSON(61.904275, 6.715641)
 

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [6.7156, 61.9043, 3]},
 'properties': {'meta': {'updated_at': '2025-03-18T08:27:05Z',
   'units': {'air_pressure_at_sea_level': 'hPa',
    'air_temperature': 'celsius',
    'cloud_area_fraction': '%',
    'precipitation_amount': 'mm',
    'relative_humidity': '%',
    'wind_from_direction': 'degrees',
    'wind_speed': 'm/s'}},
  'timeseries': [{'time': '2025-03-18T08:00:00Z',
    'data': {'instant': {'details': {'air_pressure_at_sea_level': 1027.2,
       'air_temperature': 2.3,
       'cloud_area_fraction': 93.5,
       'relative_humidity': 97.8,
       'wind_from_direction': 334.0,
       'wind_speed': 0.2}},
     'next_12_hours': {'summary': {'symbol_code': 'cloudy'}, 'details': {}},
     'next_1_hours': {'summary': {'symbol_code': 'cloudy'},
      'details': {'precipitation_amount': 0.0}},
     'next_6_hours': {'summary': {'symbol_code': 'cloudy'},
      'details': {'precipitation_amount': 0.0}}}},
   {'time': '2025

In [40]:
#Rydder manglende verdier med fillna()
def clean_weather_data(data):
    timeseries = data["properties"]["timeseries"]

    lst = []
    for entry in timeseries:
        time = entry["time"]
        details = entry["data"]["instant"]["details"]
        lst.append({
            "Time": time,
            "Temperature": details.get("air_temperature"),
            "Rain": entry["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount"),
            "Wind direction": details.get("wind_from_direction"),
            "Medium wind": details.get("wind_speed")
        })
    
    df = pd.DataFrame(lst)
    
    #gjør om "Tid" til datetime
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

    #fyller inn evt. hull (NaN) med gjennomsnitt
    df.fillna(df.mean(numeric_only=True), inplace=True)
    
    return df

clean_weather_data(make_weatherJSON(61.904275, 6.715641))

,Time,Temperature,Rain,Wind direction,Medium wind
0,2025-03-18 08:00:00+00:00,2.3,0.000000,334.0,0.2
1,2025-03-18 09:00:00+00:00,3.0,0.000000,265.8,0.8
2,2025-03-18 10:00:00+00:00,3.7,0.000000,276.8,0.5
3,2025-03-18 11:00:00+00:00,4.4,0.000000,271.3,0.9
4,2025-03-18 12:00:00+00:00,5.0,0.000000,254.4,1.6
...,...,...,...,...,...
79,2025-03-27 06:00:00+00:00,3.6,0.044231,68.6,0.7
80,2025-03-27 12:00:00+00:00,7.5,0.044231,249.3,1.2
81,2025-03-27 18:00:00+00:00,4.9,0.044231,254.4,1.1
82,2025-03-28 00:00:00+00:00,2.6,0.044231,65.8,0.8


In [41]:
#Fjerner urealistiske verdier i API-data fra Yr.
def remove_outliers(df):
    
    if "Temperature" in df.columns:
        df = df[df["Temperature"].between(-50, 50)]
    
    if "Wind direction" in df.columns:
        df = df[df["Wind direction"].between(0, 360)]
    
    if "Medium wind" in df.columns:
        df = df[df["Medium wind"].between(0, 70)]  # eksempel: maks storm/orkan
    
    if "Rain" in df.columns:
        df = df[df["Rain"].between(0, 500)]  # unngå ekstremt rare nedbørsmengder
    
    return df

df_cleaned = remove_outliers(df_cleaned)
print(df_cleaned.head())



                        Tid  Lufttemperatur  Nedbør  Vindretning  Middelvind
0 2025-03-18 08:00:00+00:00             4.0     0.0         97.0         0.6
1 2025-03-18 09:00:00+00:00             4.5     0.0        116.6         0.3
2 2025-03-18 10:00:00+00:00             5.0     0.0        163.9         0.3
3 2025-03-18 11:00:00+00:00             5.5     0.0        275.5         1.6
4 2025-03-18 12:00:00+00:00             5.9     0.0        279.6         1.9
